In [1]:
#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


In [2]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [3]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [4]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [27]:
def gen_code(llm, question, value_list):  
        #prompt = f"Generate a Python function 'run(table)' that can answer the following question using the following table! The function must return numeric float results with accuracy to two decimal places. If result value or calculation is percent calculation return format (value, 'percent'), every orher case return (value, '') . Do not generate explanation, nor example code, just the function.\n\nQuestion: {question}  \n\nTable: {table}"
        prompt = f"You are a helpful assistant to answer questions in the financial reports domain. Generate a Python function 'run(value_list)' that can answer the following question using the following JSON value list, annotated with category and metadata! The function must return a tuple (number, scale). Result number is float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. Always use categories to select the proper values for calculation. Usually year selection is necessary the proper values for calculation. Do not generate explanation, nor example code, just the function. \n\nQuestion: {question}  \n\Value list: {value_list}"
        res = llm.invoke(prompt)
        code = res.content.replace('```python','').replace('```','')
        return (prompt, code)
    
def exec_code(code, value_list):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')


In [28]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
item = devdf.iloc[0]
table = item['table']['table']
#table = table_convert.fill_table_headers(table)
value_list = table_convert.convert_table(table)


pars = [p['text'] for p in item['paragraphs']]
question = item['questions'][5]['question']
print(question)
( prompt, code) = gen_code(llm, question, value_list)
res = exec_code(code, value_list)
print(value_list, code, res,  item['questions'][5]['answer'])

What is the percentage change in Other in 2019 from 2018?
[{'number_value': 1452.4, 'scale': '', 'category': 'Fixed Price', 'header1': '2019', 'header2': 'Years Ended September 30,'}, {'number_value': 1146.2, 'scale': '', 'category': 'Fixed Price', 'header1': '2018', 'header2': 'Years Ended September 30,'}, {'number_value': 1036.9, 'scale': '', 'category': 'Fixed Price', 'header1': '2017', 'header2': 'Years Ended September 30,'}, {'number_value': 44.1, 'scale': '', 'category': 'Other', 'header1': '2019', 'header2': 'Years Ended September 30,'}, {'number_value': 56.7, 'scale': '', 'category': 'Other', 'header1': '2018', 'header2': 'Years Ended September 30,'}, {'number_value': 70.8, 'scale': '', 'category': 'Other', 'header1': '2017', 'header2': 'Years Ended September 30,'}, {'number_value': 1496.5, 'scale': '', 'category': 'Total sales', 'header1': '2019', 'header2': 'Years Ended September 30,'}, {'number_value': 1202.9, 'scale': '', 'category': 'Total sales', 'header1': '2018', 'heade

In [34]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    value_list = table_convert.convert_table(table)
    
    #docs = valuelist2docs(value_list)
    #v_vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
    #v_retriever = v_vectorstore.as_retriever()
            
    #pars = [p['text'] for p in item['paragraphs']]
    #table = transform_elements(table, remove_thousand_separators)
    #table = [remove_thousand_separators(cell) for row in table for cell in row]
    #table = [cell.replace('$', '') for row in table for cell in row]
    #print (table)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']:            
            #sel = v_retriever.invoke(question)
            #sel_txt = '[' + ','.join([i.page_content for i in sel])  + ']'

            ( prompt, code)  = gen_code(llm, q['question'], value_list)
            r = exec_code(code, value_list)
            err =  None
            if r is None:
                (pred_value, pred_scale) = ("", "")
            else:   
                if len(r) == 2:                    
                    (pred_value, pred_scale) = r    
                    if isinstance(pred_value, tuple) and len(pred_value) == 2:
                        print('$$$$')
                        (pred_value, pred_scale) = pred_value                     
                    if pred_scale == "%" or pred_scale == "percentage"  :
                        pred_scale = 'percent'
                    if pred_scale not in ["", 'thousand', 'million', 'billion', 'percent']:
                        print('Invalid ', pred_scale)
                        pred_scale = ""    
                    #if item['table']['uid'] in res_scale and pred_scale != 'percent':
                    #    pred_scale = res_scale[item['table']['uid']]
                    if isinstance(pred_value, str):
                        #print("string")
                        if  pred_value.startswith('[Error]'):
                            (pred_value, pred_scale) = ("", "")
                            err,_ = r                                                        
                        
                elif len(r) == 1:
                    (pred_value, pred_scale) = (r, "")        
                else:
                    (pred_value, pred_scale) = ("", "")        
                
                
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_scale, q, code, prompt,value_list, err))


0it [00:00, ?it/s]

[Error]'[' was never closed (<string>, line 9)
[Error]division by zero
Invalid  $’000
(3180.0, '$’000')
Invalid  $’000


KeyboardInterrupt: 

In [35]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

0.44 0.44 0.72


In [36]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    
    if pred  is None or isinstance(pred, tuple) or isinstance(pred, str):
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #type error
        #if isinstance(pred, str):
        #    print (pred, pred_scale, ' -- ' , ans['answer'])
        #if isinstance(pred, tuple):
        #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        #print('===============================================================================')
        print('{',pred, '}', ans['answer'], ans['scale'], err)
        continue
    
    #pred = round(pred, 2)
    
    if (pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit:
        good = good + 1
        #if ans['scale'] == pred_scale and  pred_scale == 'thousand':
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' == ' , ans['answer'], ans['scale'])
        
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if  ans['scale'] == 'thousand':
        #    print (q['uid'], pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
          
    else:
        #value error
        #  if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if ans['answer'] != pred and ans['scale'] == pred_scale and pred_scale != "":
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #None
        print (q['uid'],q['derivation'], pred, pred_scale, ' <> ' , ans['answer'], ans['scale'])
        print(q['question'])
        print(value_list)
        print(code)
        
    #if isinstance(pred, str):
    #    print (pred, pred_scale, ' -- ' , ans['answer'])
    #if isinstance(pred, tuple):
    #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        
print (good/len(res))

a0414f81-8dc2-44b2-a441-2c9d9c805c4d (166+178)/2 166.0 million  <>  172 million
What is the 2019 average defined contribution schemes?
[{'number_value': 166.0, 'scale': 'million', 'category': 'Defined contribution schemes', 'header1': '2019', 'header2': ''}, {'number_value': 178.0, 'scale': 'million', 'category': 'Defined contribution schemes', 'header1': '2018', 'header2': ''}, {'number_value': 192.0, 'scale': 'million', 'category': 'Defined contribution schemes', 'header1': '2017', 'header2': ''}, {'number_value': 57.0, 'scale': 'million', 'category': 'Defined benefit schemes', 'header1': '2019', 'header2': ''}, {'number_value': 44.0, 'scale': 'million', 'category': 'Defined benefit schemes', 'header1': '2018', 'header2': ''}, {'number_value': 20.0, 'scale': 'million', 'category': 'Defined benefit schemes', 'header1': '2017', 'header2': ''}, {'number_value': 223.0, 'scale': 'million', 'category': 'Total amount charged to income statement (note 23)', 'header1': '2019', 'header2': ''},

In [168]:
value_list = [{'number_value': 4.0, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2019', 'September 30,', 'Domestic']}, {'number_value': 3.75, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2018', 'September 30,', 'Domestic']}, {'number_value': 1.9, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2019', 'September 30,', 'International']}, {'number_value': 2.8, 'scale': '%', 'categories': ['Discount rate'], 'metadata': ['2018', 'September 30,', 'International']}, {'number_value': 3.4, 'scale': '%', 'categories': ['Expected return on plan assets'], 'metadata': ['2019', 'September 30,', 'International']}, {'number_value': 3.7, 'scale': '%', 'categories': ['Expected return on plan assets'], 'metadata': ['2018', 'September 30,', 'International']}]

def run(value_list):
    domestic_2019 = None
    domestic_2018 = None
    international_2019 = None
    international_2018 = None
    
    for value in value_list:
        if 'Discount rate' in value['categories']:
            if '2019' in value['metadata'] and 'Domestic' in value['metadata']:
                domestic_2019 = value['number_value']
            elif '2018' in value['metadata'] and 'Domestic' in value['metadata']:
                domestic_2018 = value['number_value']
            elif '2019' in value['metadata'] and 'International' in value['metadata']:
                international_2019 = value['number_value']
            elif '2018' in value['metadata'] and 'International' in value['metadata']:
                international_2018 = value['number_value']
    print()
    domestic_diff = abs(domestic_2019 - domestic_2018)
    international_diff = abs(international_2019 - international_2018)
    
    return (round(abs(domestic_diff - international_diff), 2), '')

run(value_list)


(0.65, '')

In [159]:
wrong_indexes = [idx for idx, m in enumerate(metrics._details) if m['f1']<0.2]
resdf = pd.DataFrame(res)
resdf2 = resdf[resdf.index.isin(wrong_indexes)]

resdf2["answer"] = resdf2[0].apply(lambda x: x['answer'])
resdf2["scale"] = resdf2[0].apply(lambda x: x['scale'])
resdf2["qid"] = resdf2[3].apply(lambda x: x['uid'])
resdf2["q"] = resdf2[3].apply(lambda x: x['question'])
resdf2

/tmp/ipykernel_290687/596054401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resdf2["answer"] = resdf2[0].apply(lambda x: x['answer'])
/tmp/ipykernel_290687/596054401.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resdf2["scale"] = resdf2[0].apply(lambda x: x['scale'])
/tmp/ipykernel_290687/596054401.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

,0,1,2,3,4,5,6,7,answer,scale,qid,q
0,"{'answer_type': 'arithmetic', 'answer': -12.6,...",,,{'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23'...,def run(value_list):\n value_dict = {}\n ...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 1452.4, 'scale': '', 'catego...",[Error]KeyError('Other'),-12.6,million,eb787966-fa02-401f-bfaf-ccabf3828b23,What is the change in Other in 2019 from 2018?
7,"{'answer_type': 'arithmetic', 'answer': 172, '...",178.67,,{'uid': 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'...,\ndef run(value_list):\n total = 0\n cou...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,172.0,million,a0414f81-8dc2-44b2-a441-2c9d9c805c4d,What is the 2019 average defined contribution ...
8,"{'answer_type': 'arithmetic', 'answer': 50.5, ...",40.33,,{'uid': 'bf7abd62-d9cd-48d2-8826-1457684019a3'...,\ndef run(value_list):\n total = 0\n cou...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,50.5,million,bf7abd62-d9cd-48d2-8826-1457684019a3,What is the 2019 average defined benefit schemes?
9,"{'answer_type': 'arithmetic', 'answer': 121.5,...",109.0,,{'uid': '4d259081-6da6-44bd-8830-e4de0031744c'...,\ndef run(value_list):\n dc_2019 = [item['n...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,121.5,million,4d259081-6da6-44bd-8830-e4de0031744c,What is the difference between 2019 average de...


In [161]:
resdf2

,0,1,2,3,4,5,6,7,answer,scale,qid,q
0,"{'answer_type': 'arithmetic', 'answer': -12.6,...",,,{'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23'...,def run(value_list):\n value_dict = {}\n ...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 1452.4, 'scale': '', 'catego...",[Error]KeyError('Other'),-12.6,million,eb787966-fa02-401f-bfaf-ccabf3828b23,What is the change in Other in 2019 from 2018?
7,"{'answer_type': 'arithmetic', 'answer': 172, '...",178.67,,{'uid': 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'...,\ndef run(value_list):\n total = 0\n cou...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,172.0,million,a0414f81-8dc2-44b2-a441-2c9d9c805c4d,What is the 2019 average defined contribution ...
8,"{'answer_type': 'arithmetic', 'answer': 50.5, ...",40.33,,{'uid': 'bf7abd62-d9cd-48d2-8826-1457684019a3'...,\ndef run(value_list):\n total = 0\n cou...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,50.5,million,bf7abd62-d9cd-48d2-8826-1457684019a3,What is the 2019 average defined benefit schemes?
9,"{'answer_type': 'arithmetic', 'answer': 121.5,...",109.0,,{'uid': '4d259081-6da6-44bd-8830-e4de0031744c'...,\ndef run(value_list):\n dc_2019 = [item['n...,Generate a Python function 'run(value_list)' t...,"[{'number_value': 166.0, 'scale': '', 'categor...",None,121.5,million,4d259081-6da6-44bd-8830-e4de0031744c,What is the difference between 2019 average de...
